In [9]:
#================================================================================
# 신규 ByteLevelBPETokenizer tokenizer vocab 생성 예제
# => tokenizer 방식중, ByteLevelBPETokenizer 를 이용하여 새롭게 tokenize vocab 을 생성하는 예제임
# => OpenAI GPT-2에서는 ByteLevelBPETokenizer , kogpt-2에서는 SetnecePieceBPETokenzer 를 사용함.
# 
# 참고자료 
# https://gist.github.com/lovit/e11c57877aae4286ade4c203d6c26a32
# https://discuss.huggingface.co/t/training-sentencepiece-from-scratch/3477/2
#
#================================================================================
# [과정] => gpt-2 tokenizer 방식
#
# 1. ByteLevelBPETokenizer 정의 후 훈련 
#     => 훈련시, corpora 목록, vocab_size, min_frequency 등을 설정함
#
# 2. 훈련한 ByteLevelBPETokenizer 저장
#     => stokenizer.save_model(OUT_PATH)
#     => 이대 저장되면, 해당 폴더에 2개의 파일이 생성됨(merges.txt, vocab.json)
#     => 생성된 merges.txt, vocab.json 을 가지고 사용해도 됨. 
#
# 3. GPT2TokenizerFast로 merges.txt, vocab.json 파일 불러옴
#     => transforer_tokenizer = GPT2TokenizerFast(vocab_file=vocab_path, merges_file=merges_path, add_prefix_space = True)
#
# 4. GPT2TokenizerFast tokenizer 저장
#     => transforer_tokenizer.save_pretrained(OUT_PATH) 
#     => 정상적으로 저장되면, 해당 폴더에 아래와 같은 파일들이 생성됨
#        tokenizer_config.json ,special_tokens_map.json,vocab.json, merges.txt, added_tokens.json, tokenizer.json 
#================================================================================

import torch
import numpy as np
import pandas as pd
import tokenizers
from tokenizers import (ByteLevelBPETokenizer,
                        CharBPETokenizer,
                        SentencePieceBPETokenizer,
                        BertWordPieceTokenizer)
# 말뭉치 지정
corpus_path = '../../korpora/kowiki_20190620/wiki_test.txt'


In [10]:
stokenizer = ByteLevelBPETokenizer(add_prefix_space = False, lowercase = False)

# 훈련
stokenizer.train(
    files = [corpus_path],
    vocab_size = 52000,  # 최대 vocab 계수 
    special_tokens = ["<cls>", "<eos>", "<mask>", "<unk>", "<pad>"],  # speical token 지정
    min_frequency = 5,   # 빈도수 
    show_progress = True,
    #limit_alphabet=10000, 
)

vocab = stokenizer.get_vocab()
print(f'vcoab 길이:{len(vocab)}')




vcoab 길이:710


In [23]:
# 테스트
sentence = '반갑습니다'
output = stokenizer.encode(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%stokenizer.decode(output.ids))

=>idx   : [598, 251, 285, 244, 173, 237, 118, 267, 294]
=>tokens: ['ë°', 'ĺ', 'ê°', 'ĳ', 'ì', 'Ĭ', 'µ', 'ëĭ', 'Īëĭ¤']
=>offset: [(0, 1), (0, 1), (1, 2), (1, 2), (2, 3), (2, 3), (2, 3), (3, 4), (3, 5)]
=>decode: 반갑습니다



In [19]:
stokenizer.decode([323, 375, 237])

'오�'

In [4]:
import os
OUT_PATH = './mytokenbpe'
os.makedirs(OUT_PATH, exist_ok=True)
stokenizer.save_model(OUT_PATH)

['./mytokenbpe/vocab.json', './mytokenbpe/merges.txt']

In [3]:
# 3. GPT2TokenizerFast로 merges.txt, vocab.json 파일 불러옴
from transformers import GPT2TokenizerFast
vocab_path = './mytoken_BPE/vocab.json'
merges_path = './mytoken_BPE/merges.txt'

tokenizer = GPT2TokenizerFast(vocab_file=vocab_path, merges_file=merges_path)

In [8]:
# 테스트
sentence = 'I LOVE YOU'
output = tokenizer.encode(sentence)
print(output)
decode = tokenizer.decode(output)
print(decode)

[45, 225, 48, 51, 58, 41, 225, 61, 51, 57]
I LOVE YOU


In [8]:
# PreTrainedTokenizerFast tokenizer 저장
import os
OUT_PATH = './mytoken_BPE'
os.makedirs(OUT_PATH, exist_ok=True)
tokenizer.save_pretrained(OUT_PATH)

('./mytoken_BPE/tokenizer_config.json',
 './mytoken_BPE/special_tokens_map.json',
 './mytoken_BPE/vocab.json',
 './mytoken_BPE/merges.txt',
 './mytoken_BPE/added_tokens.json',
 './mytoken_BPE/tokenizer.json')

In [9]:
from transformers import GPT2TokenizerFast
#OUT_PATH = 'gpt-2'
# 저장된 tokenzier 불러와서 확인
tokenizer = GPT2TokenizerFast.from_pretrained(OUT_PATH,
                                            bos_token='<cls>',
                                            eos_token='<eos>',
                                            unk_token='<unk>',
                                            pad_token='<pad>',
                                            mask_token='<mask>')

# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = tokenizer.encode(sentence)
print(output)
decode = tokenizer.decode(output)
print(decode)

[172, 306, 287, 340, 375, 237, 251, 405, 311, 432, 113, 103, 292, 263, 106, 122, 362, 18]
나는 오늘 아침밥을 먹었다.


In [ ]:
tokenizer.decode([23821, 246, 97, 167])